# Project Proposal: Group_24

In [11]:
#load Necessary Packages for modelling.
library(tidyverse)
library(repr)
library(tidymodels)
library("readxl")
install.packages("arsenal")
library(arsenal)
install.packages("qwraps2")
library(qwraps2)
# define the markup language we are working in.
# options(qwraps2_markup = "latex") is also supported.
options(qwraps2_markup = "markdown")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


Attaching package: ‘qwraps2’


The following objects are masked from ‘package:arsenal’:

    gmean, gsd




## Introduction

We have chosen a publicly-available dataset from the UCI Machine Learning Repository. The particular dataset is about dry beans and the measurements and data are collected from Selcuk University in Turkey. The dataset includes 13,611 images of grains of 7 different registered dry beans that were taken with a high-resolution camera. Using this dataset, we aim to create a classification model that can predict new observations and photos of the 7 registered dry beans given to it. 

## Preliminary Exploratory Data Analysis
### Importing Data

We use the following code to import our dataset from the web:

In [12]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00602/DryBeanDataset.zip"
download.file(url, "data.zip")
unzip("data.zip")
beans_raw <- read_excel('./DryBeanDataset/Dry_Bean_Dataset.xlsx')

### Data Cleaning

Our data is already almost tidy, all we do is change the type of the class varible from a character to a factor.

In [13]:
beans_clean <- beans_raw %>%
                mutate(Class = as_factor(Class))

### Spliting Data into Training and Testing Datasets

Next we split our data into testing and training data, with 75% of the data in the training set and 25% in the testing.

In [14]:
beans_split <- initial_split(beans_clean, prop=0.75, strata = Class)
beans_train <- training(beans_split)
beans_test <- testing(beans_split)

### Exploratory Data Analysis

In [6]:
num_obs <- nrow(beans_train)

beans_test_summmary <- beans_train %>%
                        group_by(Class) %>%
                        summarize(
                            count = n(),
                            percentage = round(n()/num_obs *100,2),
                            mean_area = mean(Area),
                            std_area = sd(Area),
                            median_area = median(Area),
                            mean_perimeter = mean(Perimeter),
                            std_perimeter = sd(Perimeter)
                            )

`summarise()` ungrouping output (override with `.groups` argument)



In [15]:
summary_data <- beans_train %>%
                select(Area, Perimeter, roundness, AspectRation, Class)
summary_table_1 <- tableby(Class~., data = summary_data)


In [55]:
summary(summary_table_1)



|                            |    SEKER (N=1509)     |   BARBUNYA (N=1019)    |     BOMBAY (N=388)      |     CALI (N=1219)      |    HOROZ (N=1439)     |     SIRA (N=1985)     |   DERMASON (N=2652)   |    Total (N=10211)     | p value|
|:---------------------------|:---------------------:|:----------------------:|:-----------------------:|:----------------------:|:---------------------:|:---------------------:|:---------------------:|:----------------------:|-------:|
|**Area**                    |                       |                        |                         |                        |                       |                       |                       |                        | < 0.001|
|&nbsp;&nbsp;&nbsp;Mean (SD) | 39858.831 (4800.095)  | 69814.333 (10500.599)  | 173399.023 (22725.713)  |  75626.878 (9353.098)  | 53690.681 (7270.334)  | 44741.456 (4617.227)  | 32186.319 (4663.129)  | 53098.283 (29209.983)  |        |
|&nbsp;&nbsp;&nbsp;Range     | 28395.000 - 61150.0

|                            |    SEKER (N=1509)     |   BARBUNYA (N=1019)    |     BOMBAY (N=388)      |     CALI (N=1219)      |    HOROZ (N=1439)     |     SIRA (N=1985)     |   DERMASON (N=2652)   |    Total (N=10211)     | p value|
|:---------------------------|:---------------------:|:----------------------:|:-----------------------:|:----------------------:|:---------------------:|:---------------------:|:---------------------:|:----------------------:|-------:|
|**Area**                    |                       |                        |                         |                        |                       |                       |                       |                        | < 0.001|
|&nbsp;&nbsp;&nbsp;Mean (SD) | 39858.831 (4800.095)  | 69814.333 (10500.599)  | 173399.023 (22725.713)  |  75626.878 (9353.098)  | 53690.681 (7270.334)  | 44741.456 (4617.227)  | 32186.319 (4663.129)  | 53098.283 (29209.983)  |        |
|&nbsp;&nbsp;&nbsp;Range     | 28395.000 - 61150.000 | 41487.000 - 115967.000 | 117034.000 - 254616.000 | 45504.000 - 116272.000 | 33006.000 - 81929.000 | 31519.000 - 63612.000 | 20420.000 - 42159.000 | 20420.000 - 254616.000 |        |
|**Perimeter**               |                       |                        |                         |                        |                       |                       |                       |                        | < 0.001|
|&nbsp;&nbsp;&nbsp;Mean (SD) |   727.356 (48.072)    |   1045.951 (91.590)    |   1585.024 (114.214)    |   1057.742 (67.143)    |   920.382 (69.478)    |   796.621 (44.950)    |   666.091 (50.206)    |   855.937 (213.724)    |        |
|&nbsp;&nbsp;&nbsp;Range     |   610.291 - 925.731   |   799.426 - 1359.763   |   1265.926 - 1985.370   |   789.770 - 1326.583   |  689.294 - 1162.588   |   668.106 - 984.282   |   524.736 - 908.265   |   524.736 - 1985.370   |        |
|**roundness**               |                       |                        |                         |                        |                       |                       |                       |                        | < 0.001|
|&nbsp;&nbsp;&nbsp;Mean (SD) |     0.945 (0.032)     |     0.800 (0.049)      |      0.865 (0.027)      |     0.847 (0.023)      |     0.794 (0.032)     |     0.884 (0.024)     |     0.908 (0.030)     |     0.873 (0.060)      |        |
|&nbsp;&nbsp;&nbsp;Range     |     0.595 - 0.991     |     0.594 - 0.932      |      0.758 - 0.950      |     0.727 - 0.920      |     0.557 - 0.921     |     0.689 - 0.954     |     0.490 - 0.967     |     0.490 - 0.991      |        |
|**AspectRation**            |                       |                        |                         |                        |                       |                       |                       |                        | < 0.001|
|&nbsp;&nbsp;&nbsp;Mean (SD) |     1.245 (0.082)     |     1.545 (0.127)      |      1.584 (0.119)      |     1.733 (0.092)      |     2.026 (0.135)     |     1.569 (0.096)     |     1.491 (0.098)     |     1.583 (0.246)      |        |
|&nbsp;&nbsp;&nbsp;Range     |     1.025 - 1.680     |     1.136 - 1.950      |      1.213 - 1.880      |     1.297 - 2.008      |     1.500 - 2.389     |     1.259 - 2.007     |     1.188 - 2.010     |     1.025 - 2.389      |        |

In [64]:
our_summary1 <-
  list("Area" =
       list("Area_median"    = ~ median(Area),
            "Area_min"       = ~ min(Area),
            "Area_max"       = ~ max(Area)),
       "Perimeter" =
       list("Perimeter_min"       = ~ min(Perimeter),
            "Perimeter_median"    = ~ median(Perimeter),
            "Perimeter_max"       = ~ max(Perimeter)),
       "roundness" =
       list("roundness_min"       = ~ min(roundness),
            "roundness_max"       = ~ max(roundness)),
       "AspectRation" =
       list("AspectRation_min"       = ~ min(AspectRation),
            "AspectRation_max"       = ~ max(AspectRation))
       )

our_summary1

$Area
$Area$Area_median
~median(Area)

$Area$Area_min
~min(Area)

$Area$Area_max
~max(Area)


$Perimeter
$Perimeter$Perimeter_min
~min(Perimeter)

$Perimeter$Perimeter_median
~median(Perimeter)

$Perimeter$Perimeter_max
~max(Perimeter)


$roundness
$roundness$roundness_min
~min(roundness)

$roundness$roundness_max
~max(roundness)


$AspectRation
$AspectRation$AspectRation_min
~min(AspectRation)

$AspectRation$AspectRation_max
~max(AspectRation)



In [65]:
whole <- summary_table(group_by(summary_data, Class), our_summary1)
whole

,SEKER (N = 1509),BARBUNYA (N = 1019),BOMBAY (N = 388),CALI (N = 1219),HOROZ (N = 1439),SIRA (N = 1985),DERMASON (N = 2652)
Area_median,3.914100e+04,6.978500e+04,1.728435e+05,7.481000e+04,5.393100e+04,4.453500e+04,3.191900e+04
Area_min,2.839500e+04,4.148700e+04,1.170340e+05,4.550400e+04,3.300600e+04,3.151900e+04,2.042000e+04
Area_max,6.115000e+04,1.159670e+05,2.546160e+05,1.162720e+05,8.192900e+04,6.361200e+04,4.215900e+04
Perimeter_min,6.102910e+02,7.994260e+02,1.265926e+03,7.897700e+02,6.892940e+02,6.681060e+02,5.247360e+02
Perimeter_median,7.204740e+02,1.045719e+03,1.588333e+03,1.055499e+03,9.219990e+02,7.943490e+02,6.651100e+02
Perimeter_max,9.257310e+02,1.359763e+03,1.985370e+03,1.326583e+03,1.162588e+03,9.842820e+02,9.082650e+02
roundness_min,5.950484e-01,5.937083e-01,7.584168e-01,7.271940e-01,5.567658e-01,6.886183e-01,4.896183e-01
roundness_max,9.906854e-01,9.319586e-01,9.501045e-01,9.200291e-01,9.210589e-01,9.541042e-01,9.666028e-01
AspectRation_min,1.024868e+00,1.135792e+00,1.212715e+00,1.297228e+00,1.499551e+00,1.259251e+00,1.188088e+00
AspectRation_max,1.679979e+00,1.950371e+00,1.879648e+00,2.008074e+00,2.388873e+00,2.006989e+00,2.010000e+00
